In [ ]:
import random
import os
import numpy as np
from datasets import load_dataset
import torch
import note_seq
import glob
import sys
sys.path.append("..")
from source.tokenizer import Tokenizer
from source.transformer import Transformer
from source.noteseqhelpers import token_sequence_to_note_sequence
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# Find all the .pt files in the models directory. Search recursively.
model_paths = glob.glob("../models/**/*.pt", recursive=True)
model_paths.sort()
print(f"Found {len(model_paths)} models.")

# Widget to select a model.
model_dropdown = widgets.Dropdown(
    options=model_paths,
    description='Model:',
    disabled=False,
)
display(model_dropdown)

In [ ]:
# Get the model path from the widget and load the model.
model_path = model_dropdown.value
assert os.path.exists(model_path), "Model path does not exist."
print(f"Loading model from {model_path}")
model = Transformer.load(model_path)

In [ ]:
dataset_id = "TristanBehrens/js-fakes-4bars"

In [ ]:
# Load the tokenizer.
tokenizer_path = os.path.join(os.path.dirname(model_path), "tokenizer.json")
assert os.path.exists(tokenizer_path), f"Tokenizer path does not exist. {tokenizer_path}"
tokenizer = Tokenizer.from_config_file(tokenizer_path)
print(tokenizer.vocabulary)

# Go from latent space to sequence of tokens.

In [ ]:
# Sample from a normal distribution.
bottleneck_shape = model.get_bottleneck_shape()
bottleneck_z = torch.randn(1, *bottleneck_shape)
print(f"bottleneck shape: {bottleneck_shape}, numbers {np.prod(bottleneck_shape)}")

for _ in range(4):
    # Create the start sequence.
    start_sequence = "PIECE_START"
    start_sequence_indices = tokenizer.encode_sequence(start_sequence)
    print(f"Start sequence: {start_sequence}")
    print(f"Start sequence indices: {start_sequence_indices}")

    result_ids = model.generate(
        decoder_ids=start_sequence_indices,
        max_new_tokens=512,
        end_token_id=tokenizer.encode_token("TRACK_END"),
        bottleneck_condition=bottleneck_z, 
        temperature=0.5,
        top_k=None
    )[0]
    #print(f"Result ids: {result_ids}")

    result_sequence = tokenizer.decode_sequence(result_ids, join=True)
    print(f"Result sequence: {result_sequence}")

    note_sequence = token_sequence_to_note_sequence(result_sequence)
    note_seq.plot_sequence(note_sequence)
    note_seq.play_sequence(note_sequence, synth=note_seq.fluidsynth)

# Test reconstruction.

In [ ]:
# Load the dataset.
split_dataset = load_dataset(dataset_id)

def random_sample():
    # Select a random sample.
    random_index = random.randint(0, len(split_dataset["test"]) - 1)
    return split_dataset["test"][random_index]["text"]

random_sample()

In [ ]:
sequence = random_sample()
encoder_ids = tokenizer.encode_sequence(sequence)
print(f"Sequence: {sequence}")
print(f"Encoder ids: {encoder_ids}")
note_sequence = token_sequence_to_note_sequence(sequence)
note_seq.plot_sequence(note_sequence)
note_seq.play_sequence(note_sequence, synth=note_seq.fluidsynth)

for _ in range(4):
    # Create the start sequence.
    start_sequence = "PIECE_START"
    start_sequence_indices = tokenizer.encode_sequence(start_sequence)
    print(f"Start sequence: {start_sequence}")
    print(f"Start sequence indices: {start_sequence_indices}")

    result_ids = model.generate(
        decoder_ids=start_sequence_indices,
        encoder_ids=encoder_ids,
        max_new_tokens=512,
        end_token_id=tokenizer.encode_token("TRACK_END"),
        temperature=0.2,
        top_k=None
    )[0]
    #print(f"Result ids: {result_ids}")

    result_sequence = tokenizer.decode_sequence(result_ids, join=True)
    print(f"Result sequence: {result_sequence}")

    note_sequence = token_sequence_to_note_sequence(result_sequence)
    note_seq.plot_sequence(note_sequence)
    note_seq.play_sequence(note_sequence, synth=note_seq.fluidsynth)

# Interpolate between two latent vectors.

In [ ]:
# Sample from a normal distribution.
bottleneck_shape = model.get_bottleneck_shape()
bottleneck_z_1 = torch.randn(1, *bottleneck_shape)
bottleneck_z_2 = torch.randn(1, *bottleneck_shape)
print(f"bottleneck shape: {bottleneck_z_1.shape}, numbers {np.prod(bottleneck_z_1.shape)}")
print(f"bottleneck shape: {bottleneck_z_2.shape}, numbers {np.prod(bottleneck_z_2.shape)}")

def slerp(z1, z2, steps):
    # Ensure that z1 and z2 are the same shape
    assert z1.shape == z2.shape, "z1 and z2 must have the same shape"
    
    # Flatten the last two dimensions for easier computations
    z1_flattened = z1.view(z1.shape[0], -1)
    z2_flattened = z2.view(z2.shape[0], -1)

    # Normalize the tensors along the last dimension
    z1_norm = z1_flattened / z1_flattened.norm(dim=-1, keepdim=True)
    z2_norm = z2_flattened / z2_flattened.norm(dim=-1, keepdim=True)

    # Dot product between the normalized tensors along the last dimension
    cos_omega = torch.sum(z1_norm * z2_norm, dim=-1)
    # Clamp values to handle numerical imprecisions
    cos_omega = torch.clamp(cos_omega, -1.0, 1.0)
    omega = torch.acos(cos_omega)
    sin_omega = torch.sin(omega)

    t_values = torch.linspace(0, 1, steps).to(z1.device)
    result = []

    for t in t_values:
        interpolate = (torch.sin((1 - t) * omega).unsqueeze(-1) / sin_omega.unsqueeze(-1)) * z1_flattened + (torch.sin(t * omega).unsqueeze(-1) / sin_omega.unsqueeze(-1)) * z2_flattened
        # Reshape to the original shape of z1 and z2
        interpolate = interpolate.view(z1.shape)
        result.append(interpolate)

    # Concatenate along a new first dimension
    return torch.stack(result)

z_list = slerp(bottleneck_z_1, bottleneck_z_2, 4)
print(f"z_list shape: {z_list.shape}")
for z in z_list:
    # Create the start sequence.
    start_sequence = "PIECE_START"
    start_sequence_indices = tokenizer.encode_sequence(start_sequence)
    print(f"Start sequence: {start_sequence}")
    print(f"Start sequence indices: {start_sequence_indices}")

    result_ids = model.generate(
        decoder_ids=start_sequence_indices,
        max_new_tokens=512,
        end_token_id=tokenizer.encode_token("TRACK_END"),
        bottleneck_condition=z, 
        temperature=1.0,
        top_k=None
    )[0]
    #print(f"Result ids: {result_ids}")

    result_sequence = tokenizer.decode_sequence(result_ids, join=True)
    print(f"Result sequence: {result_sequence}")

    note_sequence = token_sequence_to_note_sequence(result_sequence)
    note_seq.plot_sequence(note_sequence)
    note_seq.play_sequence(note_sequence, synth=note_seq.fluidsynth)
